In [48]:
function read_monkeys(filename)
    f = open(filename)
    monkey = Dict()
    monkey_list = []
    i = 0 
    worry_modifier = 1
    for line in readlines(f)
        if line == ""
            monkey["inspected"] = 0
            push!(monkey_list, copy(monkey))
        elseif occursin("Starting items:", line)
            items_line = line[18:length(line)]
            items = split(items_line, ", ")
            monkey["items"] = [parse(BigInt, string(x)) for x in items]
        elseif occursin("Operation:", line)
            operations_line = split(line[20:length(line)], " ")
            monkey["operation"] = [string(x) for x in operations_line]
        elseif occursin("Test: ", line)
            test_value = string(last(split(line, " ")))
            monkey["test"] = parse(BigInt, test_value)
            worry_modifier *= monkey["test"]
        elseif occursin("If true:", line)
            monkey[true] = parse(Int, string(last(split(line, " ")))) + 1
        elseif occursin("If false:", line)
            monkey[false] = parse(Int, string(last(split(line, " ")))) + 1  
        end
    end
    monkey["inspected"]
    push!(monkey_list, copy(monkey))
    return monkey_list, worry_modifier
end

read_monkeys (generic function with 1 method)

In [49]:
function inspect_item(monkey, item)
    operation = monkey["operation"]
    test = monkey["test"]
    value = 0
    if operation[3] == "old"
        item *= item
    elseif operation[2] == "*"
        item *= parse(Int, operation[3])
    elseif operation[2] == "+"
        item += parse(Int, operation[3])        
    end
    
    item = Int(floor(item/3))

    dest = monkey[mod(item, test) == 0]
    return dest, item
end

function throw_items(monkey, monkeys)
    for item in monkeys[monkey]["items"]
        dest, item = inspect_item(monkeys[monkey], item)
        # println("item ", item, " to ", dest)
        push!(monkeys[dest]["items"], item)
        monkeys[monkey]["inspected"] += 1
    end
    monkeys[monkey]["items"] = []
    return monkeys
end

throw_items (generic function with 1 method)

In [50]:
function inspect_item_worry(monkey, item::BigInt, worry_mod)
    operation = monkey["operation"]
    test = monkey["test"]
    value = 0
    if operation[3] == "old"
        item *= item
    elseif operation[2] == "*"
        item *= parse(BigInt, operation[3])
    elseif operation[2] == "+"
        item += parse(BigInt, operation[3])
    end
    
    item = mod(item, worry_mod)

    dest = monkey[mod(item, test) == 0]
    return dest, item
end

function throw_items_worry(monkey, monkeys, worry_mod)
    for item in monkeys[monkey]["items"]
        dest, item = inspect_item_worry(monkeys[monkey], item, worry_mod)
        # println("item ", item, " to ", dest)
        push!(monkeys[dest]["items"], item)
        monkeys[monkey]["inspected"] += 1
    end
    monkeys[monkey]["items"] = []
    return monkeys
end

throw_items_worry (generic function with 2 methods)

In [51]:
function print_monkeys(monkeys)
    for i in 1:length(monkeys)
        println("Monkey ", i-1)
        for (key, value) in monkeys[i]
            println("\t", key, " : ", value)
        end
        println()
    end
end

function print_monkeys_item(monkeys)
    for i in 1:length(monkeys)
        print("Monkey ", i, ": ")
        for item in monkeys[i]["items"]
            print("(", typeof(item), ",", item, ") ")
        end
        println()
    end
end

function print_monkeys_inspected(monkeys)
    for i in 1:length(monkeys)
        println(i, ": ", monkeys[i]["inspected"])
    end
end

print_monkeys_inspected (generic function with 1 method)

In [52]:
# Day 11 part 1

input_file = "data/day11_data.txt"
# input_file = "data/day11_test.txt"

monkeys = read_monkeys(input_file)
# print_monkeys(monkeys)

for round in 1:20
    for i in 1:length(monkeys)
        monkeys = throw_items(i, monkeys)
    end
    # println("Round ", round,": ")
    # print_monkeys_item(monkeys)
end

# print_monkeys_item(monkeys)
leaders = [0, 0]
for i in 1:length(monkeys)
    # println(i, ": ", monkeys[i]["inspected"])
    inspected =  monkeys[i]["inspected"]
    if inspected > leaders[1]
        leaders[1] = inspected
    end
    sort!(leaders)
    # println(leaders)
end

println("Res: ", leaders[1] * leaders[2])

LoadError: ArgumentError: invalid index: "items" of type String

In [53]:
# Day 11 part 2

input_file = "data/day11_data.txt"
# input_file = "data/day11_test.txt"

monkeys, worry_mod= read_monkeys(input_file)
# print_monkeys(monkeys)
ROUNDS = [1, 20, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
for round in 1:10000
    for i in 1:length(monkeys)
        monkeys = throw_items_worry(i, monkeys, worry_mod)
        # print_monkeys_item(monkeys)
    end
    # if round in ROUNDS
    #     println("Round ", round,": ")
    #     print_monkeys_item(monkeys)
    #     print_monkeys_inspected(monkeys)
    # end
end

# print_monkeys_item(monkeys)
leaders = [0, 0]
for i in 1:length(monkeys)
    println(i, ": ", monkeys[i]["inspected"])
    inspected =  monkeys[i]["inspected"]
    if inspected > leaders[1]
        leaders[1] = inspected
    end
    sort!(leaders)
    # println(leaders)
end

println("Res: ", leaders[1] * leaders[2])

1: 52166
2: 47830
3: 1938
4: 52013
Res: 2713310158
